In [110]:
# Импорты
import yaml
import os
import shutil

from PIL import Image

import pandas as pd

import torch

from torch.utils.data import Dataset, DataLoader, random_split

from torchvision.transforms import v2 as transform

from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

In [60]:
# Загрузка данных из конфига
with open(r"..\config\main.yaml") as file:
    cfg = yaml.safe_load(file)

base_path = os.path.abspath("..").replace("\\", "/")
raw_image_path = base_path + "/" + cfg["data"]["raw_img"]
raw_data_path = "../" + cfg["data"]["raw_csv"]
processed_image_path = base_path + "/" + cfg["data"]["processed_img"]
processed_data_path = "../" + cfg["data"]["processed_csv"]
class_map = cfg["data"]["class_map"]

# Предобработка данных

Загрузим аннотационый .csv файл:


In [61]:
dataset_df = pd.read_csv(raw_data_path)

По ходу EDA было выяснено, что переменные gender, age и country нам не нужны. Также нам не нужен set_id. Выбросим эти переменные:

In [62]:
# Удаляем ненужные столбцы
dataset_df = dataset_df.drop(columns=["gender", "age", "country", "set_id"])

Теперь добавим в этот файл столбцы с путями для картинок и их метками:

In [63]:
# Добавление новых столбцов в аннотационный файл

# Списки новых столбцов
raw_img_path_list = [] # Столбец путей к картинкам в сыром датасете
processed_img_path_list = [] # Столбец путей к картинкам в обработанном датасете
emotion_class_list = []  # Столбец целевой переменной

# Цикл с проходом по папкам всех людей
for human_path in os.listdir(raw_image_path):

    # Проход по всем фото внутри папки 1 человека
    for emotion_file in os.listdir(f"{raw_image_path}\{human_path}"):
        
        # Добавление информации по столбцам
        raw_img_path_list.append(f"{raw_image_path}\{human_path}\{emotion_file}")
        processed_img_path_list.append(f"{processed_image_path}\{human_path}\{emotion_file}")
        emotion_class_list.append(emotion_file.split(".")[0])

# Обновление DataFrame
dataset_df = dataset_df.reindex(range(len(raw_img_path_list))) # Меняем длину Index

dataset_df["raw_img_path"] = raw_img_path_list
dataset_df["processed_img_path"] = processed_img_path_list
dataset_df["emotion"] = emotion_class_list

# Демонстрация аннотации
dataset_df

,raw_img_path,processed_img_path,emotion
0,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,Anger
1,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,Contempt
2,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,Disgust
3,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,Fear
4,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,Happy
...,...,...,...
147,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,Fear
148,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,Happy
149,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,Neutral
150,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,Sad


Закодируем переменную класса в числа:

In [64]:
# Кодирование метки
dataset_df["emotion"] = dataset_df["emotion"].map(class_map)

dataset_df

,raw_img_path,processed_img_path,emotion
0,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,1
1,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,2
2,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,3
3,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,4
4,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,5
...,...,...,...
147,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,4
148,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,5
149,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,6
150,c:/MyProjects/Python/PyTorch/Emotions Classifi...,c:/MyProjects/Python/PyTorch/Emotions Classifi...,7


Теперь переместим нужные файлы датасета по нужному пути:

In [65]:
# Перемещение файлов

# Счётчики для статистики
copied_count = 0 # Скопировано
overwritten_count = 0 # Перезаписано
error_count = 0 # Ошибок

# Цикл с проходом по всем строкам датасета
for idx, row in dataset_df.iterrows():
    src_path = row['raw_img_path'] # Старый путь
    dst_path = row['processed_img_path'] # Новый путь
    
    try:
        # 1. Создаём директорию назначения, если её нет
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        
        # 2. Проверяем, существует ли файл уже (для статистики)
        if os.path.exists(dst_path):
            overwritten_count += 1
        else:
            copied_count += 1
        
        # 3. Копируем файл (shutil.copy2 по умолчанию ЗАМЕНЯет файл при наличии)
        shutil.copy2(src_path, dst_path)
        
    except Exception as e:
        error_count += 1
        print(f"❌ Ошибка при копировании {src_path}: {e}")

print("Копирование завершено!")
print(f"Всего файлов: {len(dataset_df)}")
print(f"Скопировано новых: {copied_count}")
print(f"Перезаписано существующих: {overwritten_count}")
print(f"Ошибок: {error_count}")

Копирование завершено!
Всего файлов: 152
Скопировано новых: 0
Перезаписано существующих: 152
Ошибок: 0


Осталось сохранить сам аннотационный файл:

In [66]:
# Сохранение аннотации
dataset_df.to_csv(processed_data_path)

# Подготовка данных для модели

Теперь, когда мы переработали датасет, можно приступить к подготовке данных для подачи в модель. Для этого надо создать класс датасета, разделить его на тренировочную и тестовую выборку, а затем подготовить DataLoader`ы.

Создадим класс датасета:

In [115]:
class ImageDataset(Dataset):

    # Инициализация
    def __init__(self, img_path_list, labels_list, transforms = None):

        # Инициализация параметров
        self.img_path_list = img_path_list
        self.labels_list = labels_list
        self.transforms = transforms

        # Полный пайплайн преобразований для изображения(пользовательсктие аугментации + обязательные преобразования для ResNet)
        if self.transforms:
            self.full_transforms = [self.transforms, EfficientNet_B0_Weights.DEFAULT.transforms()]
        else:
            self.full_transforms = [EfficientNet_B0_Weights.DEFAULT.transforms()]
    
    # Получение длины датасета
    def __len__(self):

        return len(self.img_path_list)

    # Получение экземпляра из датасета
    def __getitem__(self, idx):

        # Получение пути к картинке и ее индекса
        img_path = self.img_path_list[idx]
        label = self.labels_list[idx]

        # Загрузка картинки
        img = Image.open(img_path).convert('RGB')

        # Применение преобразований
        for transform in self.full_transforms:
            img = transform(img)

        # Возвращаем ответ
        return img, torch.tensor(label)
        


Теперь определим аугментации к данным(базовые преобразования опредлены в ResNet18_Weights.DEFAULT.transforms()):

In [116]:
augmentation_transforms = transform.Compose([
    transform.RandomHorizontalFlip(p=0.5),
    transform.RandomRotation(degrees=15),
    transform.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
])

Создадим датасет и разделим его на тренировчный и тестовый:

In [122]:
# Разделение датасета
train_indices, test_indices = random_split(range(len(dataset_df)), [0.85, 0.15], generator=torch.Generator().manual_seed(42))

# Инициализация классов датасета
train_ds = ImageDataset(dataset_df["processed_img_path"].values[train_indices], labels_list = 
    dataset_df["emotion"].values[train_indices], transforms = augmentation_transforms)

test_ds = ImageDataset(dataset_df["processed_img_path"].values[test_indices], labels_list = 
    dataset_df["emotion"].values[test_indices], transforms = None)

Осталось создать DataLoader`ы:

In [124]:
# Создание загрузчиков данных

BATCH_SIZE = 4 # Размер батча

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True) # Для обучения
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False) # Для тестирования